### Load Library

In [1]:
import pandas as pd
import json
import os
from zipfile import ZipFile
from tqdm import tqdm
import numpy as np
import re
import importlib
from unidecode import unidecode
from src import table_parser
# from src import image_parser
from src import section_parser
from src import text_parser
importlib.reload(table_parser)
importlib.reload(section_parser)
importlib.reload(text_parser)
import itertools as it
import os

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
import pandas as pd
# import fitz
import os
import time
import pandas as pd
import json
from tqdm import tqdm
import json
import chromadb
from chromadb.utils import embedding_functions
from langchain.vectorstores import Chroma
# from langchain.embeddings import HuggingFaceBgeEmbeddings
import tiktoken
# import pypdfium2 as pdfium
from io import BytesIO
# import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import pickle
# from unstructured.partition.auto import partition
from unstructured.chunking.title import chunk_by_title
# from unstructured.partition.image import partition_image
from unstructured.cleaners.core import clean,clean_non_ascii_chars,group_broken_paragraphs
from langchain.schema.document import Document
from tqdm import tqdm
# from unstructured.partition.auto import partition_image
from unstructured.chunking.title import chunk_by_title
from unstructured.cleaners.core import clean,clean_non_ascii_chars,group_broken_paragraphs
import nltk
nltk.download('punkt')  
from nltk.tokenize import sent_tokenize
import ast
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
import os.path
import zipfile
import json

# custom_text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size = 700,
#     chunk_overlap  = 0,
#     length_function = len,
#     separators = ['. ']
# )
# def chunker(sample):
#     custom_texts = custom_text_splitter.create_documents([sample])
#     custom_texts_clean = []
#     for custom_text in custom_texts:
#         temp = custom_text.page_content.strip('. ')+ "."
#         custom_texts_clean.append(temp)
#     return custom_texts_clean

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ry38682\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
### Open source embeddings
embeddings = SentenceTransformerEmbeddings(model_name="BAAI/bge-small-en-v1.5", model_kwargs = {'device': 'cpu'})
# embeddings = SentenceTransformerEmbeddings(model_name="intfloat/e5-mistral-7b-instruct", model_kwargs = {'device': 'cpu'})

### Initialize Package

In [3]:
adobe_table_parser = table_parser.AdobeTableParser()
adobe_section_parser = section_parser.AdobeSectionParser()
adobe_text_parser = text_parser.AdobeTextParser()

### Post-Process Adobe API output file "structureData.json" 

In [4]:
def parse_json_rec(section, pos):
    result = []
    result.append({
        "item": section["section_name"],
        # "id": ".".join([str(x) for x in pos])
        "id": section["id"]
    })
    for idx, item in enumerate(section["sub_section"]):
        result.extend(parse_json_rec(item, pos+[idx+1]))
    return result

def create_toc(std_json):
    toc = []
    for idx, section in enumerate(std_json):
        toc.extend(parse_json_rec(section, [idx+1]))

    return toc

In [5]:
adobe_json = json.load(open(f"adobe api output/structuredData.json", encoding = 'utf-8'))

### Create header data

In [6]:
sectionName_pageNumber = []
for idx in range(7, 45):
    toc = adobe_json['elements'][idx]
    toc_list = toc['Text'].split('.')
    sectionName_pageNumber.append([toc_list[0].strip(), toc_list[0].strip() + '_' + str(toc_list[-1].strip()) + '_' + str(toc_list[-1].strip())])
    # header_data['section_name'].append(toc_list[0])
    # header_data['page_number'].append(int(float(toc_list[-1].strip())))
    # print(toc_list[0], toc_list[-1])

In [7]:
sectionName_pageNumber[:5]

[['LUCENT INVESTIGATORS (LUCENT Study Group)',
  'LUCENT INVESTIGATORS (LUCENT Study Group)_3_3'],
 ['LUCENT Steering Committee Members',
  'LUCENT Steering Committee Members_24_24'],
 ['SUPPLEMENTAL METHODS', 'SUPPLEMENTAL METHODS_24_24'],
 ['SELECT INCLUSION CRITERIA: LUCENT-1',
  'SELECT INCLUSION CRITERIA: LUCENT-1_24_24'],
 ['SELECT EXCLUSION CRITERIA: LUCENT-1',
  'SELECT EXCLUSION CRITERIA: LUCENT-1_27_27']]

### Create text data

In [8]:
file_name = 'processed data'

adobe_json = json.load(open(f"adobe api output/structuredData.json", encoding = 'utf-8'))
print("Total objects:", len(adobe_json["elements"]))
tabular_data = pd.json_normalize(adobe_json["elements"])
table_dict = adobe_table_parser.get_tables_info_dictionary(tabular_data, file_name)
section_hierarchy = adobe_section_parser.create_section_hierarchy(adobe_json)
if not os.path.exists(f"outputs/{file_name}/"): os.makedirs(f"outputs/{file_name}/")
with open(f"outputs/{file_name}/intermediate.json", 'w') as f:
    json.dump(section_hierarchy, f, indent = 4)
intermediate2 = adobe_text_parser.get_sections_from_json(adobe_json["elements"], section_hierarchy, table_dict)
with open(f"outputs/{file_name}/intermediate2.json", "w") as f:
    json.dump(intermediate2, f, indent = 4)

toc = create_toc(intermediate2)
with open(f"outputs/{file_name}/intermediate_toc.json", "w") as f:
    json.dump(toc, f, indent = 4)

final_json = {
    "document_metadata": {
        "strategy_name": "AdobeTextStandardization",
        "strategy_version": "1.0",
        "file_name": file_name,
        "document_type": "Protocol",
        "client_name": "default",
        "confidential_flag": False,
    },
    "document_toc": toc,
    "document_content": intermediate2
}

with open(f"outputs/{file_name}/full_doc_standardized.json", "w") as f:
    json.dump(final_json, f, indent = 4)
# except Exception as e:
#     print(e)
#     pass

Total objects: 4543


In [9]:
json_file = json.load(open("outputs/processed data/full_doc_standardized.json"))

In [10]:
json_file.keys()

dict_keys(['document_metadata', 'document_toc', 'document_content'])

In [11]:
section_hierarchy['parent'].keys()

dict_keys(['//Document/Title', '//Document/P', '//Document/P[2]', '//Document/P[3]', '//Document/H1', '//Document/H1[2]', '//Document/H1[3]', '//Document/H1[4]', '//Document/H1[5]', '//Document/H1[6]', '//Document/H1[7]', '//Document/H1[8]'])

### Extract Tabular data

In [12]:
tabular_df =table_dict

In [13]:
tabular_df.keys()

dict_keys(['//Document/Table', '//Document/Table[2]', '//Document/Table[3]', '//Document/Table[4]', '//Document/Table[5]', '//Document/Table[6]', '//Document/Table[7]', '//Document/Table[8]', '//Document/Table[9]', '//Document/Table[10]', '//Document/Table[11]', '//Document/Table[12]', '//Document/Table[13]', '//Document/Table[14]', '//Document/Table[15]', '//Document/Table[16]', '//Document/Table[17]', '//Document/Table[18]', '//Document/Table[19]'])

In [14]:
tabular_df['//Document/Table'].keys()

dict_keys(['section_type', 'text_details', 'image_details', 'table_details', 'start_page', 'end_page', 'id'])

In [15]:
tabular_df['//Document/Table[9]']['table_details']['label']

' SUPPLEMENTARY TABLES '

In [16]:
tabular_chunk_dict = []
for key in tabular_df.keys():
    text_data = ''
    start = tabular_df[key]['start_page']
    end = tabular_df[key]['end_page']
    header = tabular_df[key]['table_details']['label'].strip()
    for dictt in tabular_df[key]['table_details']['body']:
        if dictt['col_id'] == 0:
            text_data += dictt['value']
            # print(dictt['value'])
    if text_data != '':
        tabular_chunk_dict.append([text_data, header + '_' + str(start)+'_'+str(end)])

In [17]:
print(tabular_chunk_dict[12])

["TEAE, n (%) SAE, n (%) Discontinuation due to AE, n (%) Death, n (%) Common TEAEs, n (%)a Arthralgia Nasopharyngitis Ulcerative colitis Anemia Headache Diarrhoea Injection site pain AEs of interest Infections: All Infections: Serious Infections: Opportunisticb Adjudicated cerebrocardiovascular events Malignanciesc Depressiond Suicide/self-injury Hepatic-related Immediate hypersensitivity reactionse Infusion/injection site reactionsf TEAE=treatment-emergent adverse events; SAE=serious adverse events; AE=adverse events.*One death occurred during the post-treatment follow-up period due to COVID-19 infection after patient discontinued from the study and withdrew consent. a: ≥3% in any treatment group; by decreasing frequency at LUCENT-2 Week 12; b: Once case of gastrointestinal candidiasis and one of cytomegalovirus oesophagitis at Week 12; One case of herpes simplex at Week 40; c: Two Week 12 malignancies were squamous cell carcinoma, one was adenocarcinoma of the colon, and one was rec

In [18]:
print(len(tabular_chunk_dict))

15


### Create Embedding for tabular + Header data

In [19]:
header_tabular_list = sectionName_pageNumber + tabular_chunk_dict
print(len(header_tabular_list))

53


In [23]:
docs_list = []
for idx in range(len(header_tabular_list)):
    chunk = header_tabular_list[idx][0]
    meta_data = header_tabular_list[idx][1]
    docs_list.extend([Document(page_content=chunk, metadata={'header_pageNumber':meta_data})]) 

In [29]:
len(docs_list)

53

In [35]:
# db = Chroma.from_documents(docs_list, embeddings, collection_name = 'Chromadb_only_header_table_v4', persist_directory = 'Chromadb_only_header_table_v4/')
# db.persist()
# print(len(db.get()['ids']))

In [36]:
header_db = Chroma(collection_name = 'Chromadb_only_header_table_v4', persist_directory="Chromadb_only_header_table_v4/", embedding_function = embeddings)

In [37]:
header_retriever = header_db.as_retriever( search_kwargs ={
                                                   "k": 3,
                                               })

# query = """
# How does Mirikizumab's safety and efficacy vary based on patient's geography?
# """


query = """
What are all the primary and secondry outcomes of LUCENT1 and LUCENT2
"""

# query = """
# What are baseline demographics and disease characteristics across patient subgroups by geography  
# """

retrieved_docs = header_retriever.get_relevant_documents(query)
full_context = ''
for doc in retrieved_docs:
    full_context += doc.page_content
    print(doc.metadata)
    print('\t', doc.page_content, '\n' )
    full_context += '\n\n'
# print(full_context)

{'header_pageNumber': 'Table S6: Primary and key secondary outcomes of LUCENT-1 and LUCENT-2 by biologic failure profile_59_59'}
	 Table S6: Primary and key secondary outcomes of LUCENT-1 and LUCENT-2 by biologic failure profile 

{'header_pageNumber': 'Table S1: Primary and major secondary outcomes and definitions for LUCENT-1 and LUCENT-2_52_52'}
	 Table S1: Primary and major secondary outcomes and definitions for LUCENT-1 and LUCENT-2 

{'header_pageNumber': 'Figure S6: Primary and key secondary outcomes of LUCENT-1 and LUCENT-2 by biologic or tofacitinib failure profile_45_45'}
	 Figure S6: Primary and key secondary outcomes of LUCENT-1 and LUCENT-2 by biologic or tofacitinib failure profile 



In [143]:
def format_list_dict(list_dict):
    result = ""
    current_row_id = None

    for item in list_dict:
        if current_row_id is None or current_row_id != item['row_id']:
            # Change of row_id, add a newline
            if result:
                result += '\n'
            current_row_id = item['row_id']
        else:
            # Same row_id, add a separator
            result += '--'

        # Add the 'value' key to the result
        if len(item['value']) > 0:
            result += item['value']
        else:
            result += 'EMPTY'

    return result

In [144]:
# def convert_element_to_text(element):
#     text = ""
#     if "text_details" in element and "text" in element["text_details"]:
#         text += str(element["text_details"]["text"])
#     elif "image_details" in element and len(element["image_details"]):
#         text+= "\nIMAGE HERE"
#     elif "table_details" in element and len(element["table_details"]):
#         print(element.keys())
#         # text+= "\nTABLE HERE"
#         text += "\n" + str(format_list_dict(element['table_details']['body']))
#     return text

# def iterate_rec(sub_json):
#     text_list = []
#     text_list.append(((sub_json["id"].split("|")[1] + " " + sub_json["section_name"]), int(sub_json["id"].split("|")[-1])))
#     for element in sub_json["section_data"]:
#         text = convert_element_to_text(element)
#         text_list.append(('Page_Number:'+str(element['start_page'])+'_'+str(element['end_page'])+ ': '+text, int(element["id"].split("|")[-1])))
    
#     for element in sub_json["sub_section"]:
#         text_list.extend(iterate_rec(element))
    
#     return text_list

# def convert_to_text(full_text_json):
#     text = []
#     for section in full_text_json["document_content"]:
#         text.extend(iterate_rec(section))

#     text = sorted(text, key = lambda x: x[-1])
#     text = [str(x[0]) for x in text]
#     text = "\n".join(text)
#     # with open("to_text.txt", "w") as f:
#     #     f.write(text)
#     return text



def convert_element_to_text(element):
    text = ""
    if "text_details" in element and "text" in element["text_details"]:
        text += str(element["text_details"]["text"])
    elif "image_details" in element and len(element["image_details"]):
        text+= " " ##"\nIMAGE HERE"
    elif "table_details" in element and len(element["table_details"]):
        text += "\n" + str(format_list_dict(element['table_details']['body']))
    return text

def iterate_rec(sub_json):
    text_list = []
    text_list.append(((sub_json["section_name"]), int(sub_json["id"].split("|")[-1])))
    for element in sub_json["section_data"]:
        text = convert_element_to_text(element)
        text_list.append((text, (element['start_page'], element['end_page']), int(element["id"].split("|")[-1])))
    
    for element in sub_json["sub_section"]:
        text_list.extend(iterate_rec(element))
    
    return text_list

def convert_to_text(full_text_json):
    text = []
    for section in full_text_json["document_content"]:
        text.extend(iterate_rec(section))

    # text = sorted(text, key = lambda x: x[-1])
    # text = [str(x[0]) for x in text]
    # text = "\n".join(text)
    # with open("to_text.txt", "w") as f:
    #     f.write(text)
    return text

In [145]:
full_text_file = convert_to_text(json_file)

In [146]:
print(full_text_file[:30])

[('', 1), ('Supplement to: D’Haens G, Dubinsky M, Kobayashi T, et al. Mirikizumab as induction and maintenance therapy for ulcerative colitis. N Engl J Med 2023;388:2444-55. DOI: 10.1056/NEJMoa2207940 \n\n', (0, 0), 2), ('This appendix has been provided by the authors to give readers additional information about the work. \n\n', (0, 0), 3), ('(PDF updated August 4, 2023) \n\n', (0, 0), 4), ('Supplementary Appendix ', 5), ('TABLE OF CONTENTS \n\n', (1, 1), 6), ('TABLE OF CONTENTS', 7), ('{}', (1, 2), 8), ('LUCENT INVESTIGATORS (LUCENT Study Group) ', 9), ('Sonja Gassner; Universitätsklinikum Salzburg, Müllner Hauptstraße 50, Salzburg Salzburg 5020, Austria \n\n', (3, 3), 10), ('Walter Reinisch; Medical University of Vienna, Waehringer Guertel 18-20, Vienna Vienna 1090, Austria \n\n', (3, 3), 11), ('Robert Koch; Medizinische Universität Innsbruck, Anichstraße 35, Innsbruck 6020, Austria \n\n', (3, 3), 12), ('Marc Ferrante; Universitaire Ziekenhuizen Leuven - Campus Gasthuisberg, Herestra

In [147]:
full_text_file[6]

('TABLE OF CONTENTS', 7)

In [148]:
processed_file = {}
for ele in full_text_file:
    if len(ele) == 2:
        header = ele[0].split('_')[-1]
        processed_file[header] = {'text':[], 'page_number':[]}
        # print(header)
    else:
        processed_file[header]['text'].append(ele[0])
        processed_file[header]['page_number'].append(ele[1])

# for key in processed_file.keys():
#     # print(key)
#     if len(processed_file[key]['page_number']) > 0:
#         page_no = str(processed_file[key]['page_number'][0].split('_')[0]) + '_' + str(processed_file[key]['page_number'][-1].split('_')[-1])
#         processed_file[key]['page_number'] = page_no
#     else:
#         page_no = '-1'
#         processed_file[key]['page_number'] = page_no

In [149]:
display(processed_file.keys())

dict_keys(['', 'Supplementary Appendix ', 'TABLE OF CONTENTS', 'LUCENT INVESTIGATORS (LUCENT Study Group) ', 'LUCENT Steering Committee Members ', 'SUPPLEMENTAL METHODS ', 'SELECT INCLUSION CRITERIA: LUCENT-1 ', 'Disease-Specific Inclusion Criteria ', 'Prior Medication Failure Criteria ', 'Documentation of dose, frequency, route of administration and duration of the prior failed treatment is required. ', 'Investigators must be able to document an adequate clinical trial of the medication. Patients should fulfill 1 of the following criteria: ', 'UC Medication Dose Stabilization Criteria ', 'SELECT EXCLUSION CRITERIA: LUCENT-1 ', 'Gastrointestinal Exclusion Criteria: ', 'Criteria for Prohibited Medications ', 'SELECT INCLUSION CRITERIA: LUCENT-2 ', 'SELECT EXCLUSION CRITERIA: LUCENT-2 ', 'Gastrointestinal Exclusion Criteria ', 'MAYO SCORE ', 'ENDOSCOPY METHODS ', 'Endoscopic Biopsies ', 'Geboes Scoring ', 'URGENCY NUMERIC RATING SCALE ', 'INFLAMMATORY BOWEL DISEASE QUESTIONNAIRE (IBDQ) '

In [150]:
# processed_file['LUCENT INVESTIGATORS (LUCENT Study Group) ']['text']

In [151]:
processed_file['']['page_number']

[(0, 0), (0, 0), (0, 0)]

In [152]:
# len(processed_file['LUCENT INVESTIGATORS (LUCENT Study Group) ']['page_number'])

In [153]:
def get_chunk(header, text_list, page_no_list):
    chunk_dict = {'chunk': [], 'meta_data': []}
    tmp_text = ''
    pg_no = []
    # print(text_list)
    # print(page_no_list)
    for idx in range(len(text_list)):
        tmp_text += text_list[idx] + ' '
        pg_no.append(page_no_list[idx])
        if len(tmp_text) > 800:
            chunk_dict['chunk'].append(tmp_text)
            m_dt = header + '_' + str(pg_no[0][0]) + '_' + str(pg_no[-1][1])
            chunk_dict['meta_data'].append(m_dt)
            tmp_text = ''
            pg_no = []

    if len(pg_no) > 0:
        chunk_dict['chunk'].append(tmp_text)
        m_dt = header + '_' + str(pg_no[0][0]) + '_' + str(pg_no[-1][1])
        chunk_dict['meta_data'].append(m_dt)
    
    return chunk_dict



In [37]:
final_chunk_dict = {}
for header in processed_file.keys():
    text_list = processed_file[header]['text']
    page_no_list = processed_file[header]['page_number']
    chunk_dict = get_chunk(header, text_list, page_no_list)
    final_chunk_dict[header] = chunk_dict

In [38]:
final_chunk_dict['A ']

{'chunk': ['\n<>-- Risk Difference % (95% CI) -- Pbo n (%) -- Miri n (%) -- Diff % \nAll Patients -- <>-- 39 (13.3) -- 210 (24.2) -- 10.9 \nPooled Age Group 1 -- <>-- <>-- <>-- <>\n<65 -- <>-- 36 (13.1) -- 199 (25.0) -- 11.9 \n≥65 -- <>-- 3 (15.8) -- 11 (15.3) -- <>\nPooled Age Group 2 -- <>-- <>-- <>-- <>\n<40 -- <>-- 23 (15.5) -- 109 (27.7) -- 12.2 \n≥40 -- <>-- 16 (11.0) -- 101 (21.3) -- 10.3 \nSex -- <>-- <>-- <>-- <>\nMale -- <>-- 20 (12.1) -- 105 (19.8) -- 7.7 \nFemale -- <>-- 19 (14.7) -- 105 (31.1) -- 16.3 \nEthnicity -- <>-- <>-- <>-- <>\nHispanic/Latino -- <>-- 1 (8.3) -- 7 (21.9) -- <>\nNon-Hispanic/Non-Latino -- <>-- 33 (14.8) -- 155 (24.8) -- 10.0 \nRace -- <>-- <>-- <>-- <>\nAmerican Indian/Alaska Native -- <>-- 2 (100.0) -- 2 (20.0) -- <>\nAsian -- <>-- 8 (11.8) -- 63 (28.3) -- 16.5 \nBlack/African American -- <>-- 0 (0.0) -- 2 (20.0) -- <>\nWhite -- <>-- 28 (12.8) -- 142 (23.1) -- 10.3 \nMultiple -- <>-- 1 (50.0) -- 0 (0.0) -- <>\nGeographic Region 1 -- <>-- <>-- <>-- <

In [39]:
for key in final_chunk_dict.keys():
    print(key)


Supplementary Appendix 
TABLE OF CONTENTS
LUCENT INVESTIGATORS (LUCENT Study Group) 
LUCENT Steering Committee Members 
SUPPLEMENTAL METHODS 
SELECT INCLUSION CRITERIA: LUCENT-1 
Disease-Specific Inclusion Criteria 
Prior Medication Failure Criteria 
Documentation of dose, frequency, route of administration and duration of the prior failed treatment is required. 
Investigators must be able to document an adequate clinical trial of the medication. Patients should fulfill 1 of the following criteria: 
UC Medication Dose Stabilization Criteria 
SELECT EXCLUSION CRITERIA: LUCENT-1 
Gastrointestinal Exclusion Criteria: 
Criteria for Prohibited Medications 
SELECT INCLUSION CRITERIA: LUCENT-2 
SELECT EXCLUSION CRITERIA: LUCENT-2 
Gastrointestinal Exclusion Criteria 
MAYO SCORE 
ENDOSCOPY METHODS 
Endoscopic Biopsies 
Geboes Scoring 
URGENCY NUMERIC RATING SCALE 
INFLAMMATORY BOWEL DISEASE QUESTIONNAIRE (IBDQ) 
PHARMACOKINETICS 
SUMMARY OF ECOA TRANSCRIPTION ERRORS 
Rectal Bleeding (RB) elec

In [158]:
final_chunk_dict['A ']

{'chunk': ['\n<>-- Risk Difference % (95% CI) -- Pbo n (%) -- Miri n (%) -- Diff % \nAll Patients -- <>-- 39 (13.3) -- 210 (24.2) -- 10.9 \nPooled Age Group 1 -- <>-- <>-- <>-- <>\n<65 -- <>-- 36 (13.1) -- 199 (25.0) -- 11.9 \n≥65 -- <>-- 3 (15.8) -- 11 (15.3) -- <>\nPooled Age Group 2 -- <>-- <>-- <>-- <>\n<40 -- <>-- 23 (15.5) -- 109 (27.7) -- 12.2 \n≥40 -- <>-- 16 (11.0) -- 101 (21.3) -- 10.3 \nSex -- <>-- <>-- <>-- <>\nMale -- <>-- 20 (12.1) -- 105 (19.8) -- 7.7 \nFemale -- <>-- 19 (14.7) -- 105 (31.1) -- 16.3 \nEthnicity -- <>-- <>-- <>-- <>\nHispanic/Latino -- <>-- 1 (8.3) -- 7 (21.9) -- <>\nNon-Hispanic/Non-Latino -- <>-- 33 (14.8) -- 155 (24.8) -- 10.0 \nRace -- <>-- <>-- <>-- <>\nAmerican Indian/Alaska Native -- <>-- 2 (100.0) -- 2 (20.0) -- <>\nAsian -- <>-- 8 (11.8) -- 63 (28.3) -- 16.5 \nBlack/African American -- <>-- 0 (0.0) -- 2 (20.0) -- <>\nWhite -- <>-- 28 (12.8) -- 142 (23.1) -- 10.3 \nMultiple -- <>-- 1 (50.0) -- 0 (0.0) -- <>\nGeographic Region 1 -- <>-- <>-- <>-- <

In [159]:
docs_list = []
for key in final_chunk_dict.keys():
    chunk_list = final_chunk_dict[key]['chunk']
    meta_data_list = final_chunk_dict[key]['meta_data']
    for idx in range(len(chunk_list)):
        chunk = chunk_list[idx]
        meta_data = meta_data_list[idx]
        docs_list.extend([Document(page_content=chunk, metadata={'header_pageNumber':meta_data})]) 

In [165]:
len(docs_list)

120

### Create ChromaDB vector database 

In [166]:
db = Chroma.from_documents(docs_list, embeddings, collection_name = 'Chromadb_open_src_emb_pdf_v3', persist_directory = 'Chromadb_open_src_emb_pdf_v3/')
db.persist()
print(len(db.get()['ids']))

120


In [167]:
db = Chroma(collection_name = 'Chromadb_open_src_emb_pdf_v3', persist_directory="Chromadb_open_src_emb_pdf_v3/", embedding_function = embeddings)

In [168]:
retriever = db.as_retriever( search_kwargs ={
                                                   "k": 5,
                                               })

In [199]:
def get_answer(query):
    retrieved_docs = retriever.get_relevant_documents(query)
    page_no_list = []
    full_context = ''
    for doc in retrieved_docs:
        full_context += doc.page_content
        header, start, end = str(doc.metadata['header_pageNumber']).split('_')
        if int(float(start)) == int(float(end)):
            page_no_list.append(int(float(start)))
        else:
            page_no_list.append(int(float(start)))
            page_no_list.append(int(float(end)))

        print(doc.metadata)
        print(len(doc.page_content))
        # print('\t', doc.page_content, '\n' )
        full_context += '\n\n'
        print('*'*50)
    return page_no_list
# print(full_context)

In [200]:
# query = """
# How does Mirikizumab's safety and efficacy vary based on patient's geography?
# """


# query = """
# What are all the primary and secondry outcomes of LUCENT1 and LUCENT2
# """

query = """
What are baseline demographics and disease characteristics across patient subgroups by geography  
"""
get_answer(query)

{'header_pageNumber': 'SUPPLEMENTARY TABLES _57_57.0'}
2742
**************************************************
{'header_pageNumber': 'B. _42.0_42.0'}
4608
**************************************************
{'header_pageNumber': 'A _39.0_39.0'}
1794
**************************************************
{'header_pageNumber': 'B. _41.0_41.0'}
1011
**************************************************
{'header_pageNumber': 'STATISTICAL METHODOLOGY _35_35'}
931
**************************************************


[57, 42, 39, 41, 35]

In [186]:
# def display_pdf_page(pdf_path, page_number):
#     try:
#         # Open the PDF file
#         pdf_document = fitz.open(pdf_path)

#         # Get the specified page
#         page = pdf_document[page_number - 1]

#         # Display the page
#         pdf_document[page_number - 1].show()

#         # Close the PDF document
#         pdf_document.close()

#     except Exception as e:
#         print(f"Error: {e}")

import fitz  # PyMuPDF
from PIL import Image

def display_pdf_page(pdf_path, page_number):
    try:
        # Open the PDF file
        pdf_document = fitz.open(pdf_path)

        # Get the specified page
        page = pdf_document[page_number]

        # Render the page as an image
        pix = page.get_pixmap()
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

        # Display the image using the default image viewer
        img.show()

        # Close the PDF document
        pdf_document.close()

    except Exception as e:
        print(f"Error: {e}")

In [187]:
pdf_path = 'nejmoa2207940_appendix.pdf'
page_number = 39
display_pdf_page(pdf_path, page_number)

In [178]:
pdf_document = fitz.open(pdf_path)

In [183]:
pdf_document.ShownPages

{}

#### Other way

In [27]:
import pandas as pd
import json

df = json.load(open('structuredData.json', 'r'))

In [28]:
df2 = pd.json_normalize(df['elements']) 

In [30]:
filtered_df = df2[['Page', 'Path', 'Text', 'attributes.ColIndex', 'attributes.ColSpan', 'attributes.NumCol', 'attributes.RowIndex', 'attributes.RowSpan', 'attributes.NumRow']].copy()
# df2.head()

In [31]:
filtered_df.head(10)

,Page,Path,Text,attributes.ColIndex,attributes.ColSpan,attributes.NumCol,attributes.RowIndex,attributes.RowSpan,attributes.NumRow
0,0.0,//Document/Title,Supplementary Appendix,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,//Document/P,"Supplement to: D’Haens G, Dubinsky M, Kobayash...",NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,//Document/P[2],This appendix has been provided by the authors...,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,//Document/P[3],"(PDF updated August 4, 2023)",NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,//Document/H1,Supplementary Appendix,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,//Document/P[4],TABLE OF CONTENTS,NaN,NaN,NaN,NaN,NaN,NaN
6,1.0,//Document/H1[2],Contents,NaN,NaN,NaN,NaN,NaN,NaN
7,1.0,//Document/TOC/TOCI/Span,LUCENT INVESTIGATORS (LUCENT Study Group) .......,NaN,NaN,NaN,NaN,NaN,NaN
8,1.0,//Document/TOC/TOCI[2]/Span,LUCENT Steering Committee Members................,NaN,NaN,NaN,NaN,NaN,NaN
9,1.0,//Document/TOC/TOCI[3]/Span,SUPPLEMENTAL METHODS.............................,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
print(list(filtered_df['Path'][:100]))

['//Document/Title', '//Document/P', '//Document/P[2]', '//Document/P[3]', '//Document/H1', '//Document/P[4]', '//Document/H1[2]', '//Document/TOC/TOCI/Span', '//Document/TOC/TOCI[2]/Span', '//Document/TOC/TOCI[3]/Span', '//Document/TOC/TOCI[4]/Span', '//Document/TOC/TOCI[5]/Span', '//Document/TOC/TOCI[6]/Span', '//Document/TOC/TOCI[7]/Span', '//Document/TOC/TOCI[8]/Span', '//Document/TOC/TOCI[9]/Span', '//Document/TOC/TOCI[10]/Span', '//Document/TOC/TOCI[11]/Span', '//Document/TOC/TOCI[12]/Span', '//Document/TOC/TOCI[13]/Span', '//Document/TOC/TOCI[14]/Span', '//Document/TOC/TOCI[15]/Span', '//Document/TOC/TOCI[16]/Span', '//Document/TOC/TOCI[17]/Span', '//Document/TOC/TOCI[18]/Span', '//Document/TOC/TOCI[19]/Span', '//Document/TOC/TOCI[20]/Span', '//Document/TOC/TOCI[21]/Span', '//Document/TOC/TOCI[22]/Span', '//Document/TOC/TOCI[23]/Span', '//Document/TOC/TOCI[24]/Span', '//Document/TOC/TOCI[25]/Span', '//Document/TOC/TOCI[26]/Span', '//Document/TOC/TOCI[27]/Span', '//Document/TOC/T